In [1]:
import random
import json
import pickle
import numpy as np

import nltk 
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet 

nltk.download('punkt')
nltk.download('wordnet')

import tensorflow
import tflearn
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD

import visualkeras

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\PC\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\PC\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Instructions for updating:
non-resource variables are not supported in the long term
curses is not supported on this machine (please install/reinstall curses for an optimal experience)


ModuleNotFoundError: No module named 'visualkeras'

In [ ]:
lemmatizer = WordNetLemmatizer()

In [ ]:
intents = json.loads(open('intents.json').read())

print(intents)

In [ ]:
words= []
classes=[]
documents=[]
ignore_letters = ['?','!','.',',']

In [ ]:
for intent in intents['intents']:
    for pattern in intent['patterns']:
        word_list = nltk.word_tokenize(pattern)
        words.extend(word_list)
        documents.append((word_list,intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [ ]:
words = [lemmatizer.lemmatize(word) for word in words if word not in ignore_letters]
words = sorted(list(set(words)))

classes = sorted(list(set(classes)))

print(len(documents), "documents")
print(len(classes), "classes", classes)
print(len(words), "unique stemmed words", words)

In [ ]:
training = []
output=[]
output_empty = [0]* len(classes)

for x,doc in enumerate(documents):
    bag = []
    word_patterns= doc[0]
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in word_patterns]
    for word in words:
        bag.append(1) if word in word_patterns else bag.append(0)
    
    output_row = output_empty[:]
    index = classes.index(doc[1])
    output_row[index] =1
    training.append([bag,output_row])
    
    
training = np.array(training)

train_x = list(training[:,0])
train_y = list(training[:,1])

In [ ]:
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)

In [ ]:
def wordcheck(AI_word,inplst):
    checklst=[]
    for i in range(len(inplst)):
        if inplst[i].lower()==AI_word.lower():
            checklst.append(True)
        else:
            checklst.append(False)
    return any(checklst)
def synonymCheck(AI_word,inplst):
    synlst=[]
    checklst=[]
    for vsyn in wordnet.synsets(AI_word):
        for l in vsyn.lemmas():
            synlst.append(l.name())
    for i in range(len(inplst)):
        for j in range(len(synlst)):
            if (inplst[i].lower()==synlst[j].lower()):
                checklst.append(True)
            else:
                checklst.append(False)
    return any(checklst)


def inputter(AIList):
    outlst=[] #out list of length len(AIList)
    inputstr=input("Ask a question:")
    inputlst1=nltk.word_tokenize(inputstr)
    inputlst=[lemmatizer.lemmatize(word.lower()) for word in inputlst1]
    for AIwordIndex in range(len(AIList)):
        if (wordcheck(AIList[AIwordIndex],inputlst)==True) or (synonymCheck(AIList[AIwordIndex],inputlst)==True):
            outlst.append(1)
        else:
            outlst.append(0)
    return outlst

ai_data = inputter(words)
ai_data_np = np.array([ai_data])


In [ ]:
visualkeras.layered_view(model)

In [ ]:

results = model.predict(ai_data_np)[0]
results = [[i,r] for i,r in enumerate(results)]
# sort by strength of probability
results.sort(key=lambda x: x[1], reverse=True)
return_list = []
for r in results:
    return_list.append((classes[r[0]], str(r[1])))
    # return tuple of intent and probability
print(results)

In [ ]:
tag=classes[results[0][0]]
print(tag)
tag_index= tag.split("_")
tag_index=int(tag_index[0])
responses = intents['intents'][tag_index-1]['responses']
print(random.choice(responses))